In [1]:
%matplotlib inline
from keras.preprocessing.image import load_img, img_to_array,array_to_img,ImageDataGenerator
import matplotlib.pyplot as Plt
import keras
from keras import metrics
from keras.models import Sequential,Model,load_model,model_from_json
from keras.layers import Dense,Activation,Input
from keras.layers import Conv2D, Flatten
from keras.layers import MaxPooling2D,Dropout
import numpy as np
import json
import pprint
from sklearn.cross_validation import train_test_split
from keras.applications.vgg16 import VGG16


Using TensorFlow backend.
/usr/local/src/pyenv/versions/latest_mod/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
model=load_model("model.h5")
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 480, 640, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 480, 640, 64)  1792                                         
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 480, 640, 64)  36928                                        
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 240, 320, 64)  0                                            
___________________________________________________________________________________________

In [8]:

k=1367
width=480
height=640

gominum=7
xywhc=5
batch_size=1
epochs=10



In [5]:

x=np.load("x.npy")
y=np.load("y.npy")
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)


y_xywh_train=np.zeros((len(y_train),gominum,4))
y_c_train=np.zeros((len(y_train),gominum))
for i in range(len(y_train)):
    for j in range(len(y_train[0])):
        y_xywh_train[i][j]=y_train[i][j][:4]
        if(y_train[i][j][4]==1):
            y_c_train[i][j]=1
            

y_xywh_test=np.zeros((len(y_test),gominum,4))
y_c_test=np.zeros((len(y_test),gominum))                      
for i in range(len(y_test)):
    for j in range(len(y_test[0])):
        y_xywh_test[i][j]=y_test[i][j][:4]
        if(y_test[i][j][4]==1):
            y_c_test[i][j]=1
 


x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255
#y_train=y_train.reshape(-1,35)
#y_test=y_test.reshape(-1,35)
y_xywh_train=y_xywh_train.reshape(-1,28)/255
y_xywh_test=y_xywh_test.reshape(-1,28)/255

y_c_train=y_c_train.reshape(-1,7)
y_c_test=y_c_test.reshape(-1,7)


In [6]:
model.compile(optimizer="sgd",loss={'main_output': 'mean_squared_error', 'side_output': 'binary_crossentropy'},
              metrics={'main_output': 'mae', 'side_output': 'accuracy'})

In [9]:
model.fit(x=x_train,y=[y_xywh_train,y_c_train],batch_size=batch_size,epochs=epoch,verbose=1,
          validation_data=(x_test,[y_xywh_test,y_c_test]))
model.evaluate(x_test,[y_xywh_test,y_c_test],verbose=1)



NameError: name 'epoch' is not defined

In [ ]:
json_string = model.to_json()
open('model.json', 'w').write(json_string)
model.save('model.h5') 
model.save_weights('param.hdf5')